In [ ]:
    import os
    import sys
    import time
    import torch
    import numpy as np
    import matplotlib.pyplot as plt

In [ ]:
    #sys.path.insert(1, os.path.join("C:\\", "Users", "Soham", "Git", "spectral-NN", "source_codes"))
    sys.path.insert(1, os.path.join("/home", "soham", "Git", "spectral-NN", "source_codes"))
    
    import SpectralNetworks as spectNN
    import Important_functions as Ifn
    import spectral_NN_setup as setup

In [ ]:
    dirc = setup.directory
#    replicates = setup.replicates
    repl = 1

In [ ]:
        print('Example'+str(repl+1)+':')
        u = np.loadtxt(dirc+"locations.dat",dtype="float32")
        if len(u.shape)==1:
            D, d = len(u), 1
            u = u.reshape(D,1)
        else:
            D, d = u.shape
        u = torch.from_numpy(u)
        x = np.loadtxt(dirc+'Example'+str(repl+1)+'.dat',dtype='float32')
        N = x.shape[0]
        if x.shape[1] != D:
            exit('Data shape mismatch!! Aborting..')
        print('N='+str(N)+', D='+str(D)+', d='+str(d))

        x = torch.from_numpy(x)
        x = x - torch.mean(x,dim=0,keepdim=True)

In [ ]:
        method = "deep"
        M, L = 15, 15
        depth, width = 3, 20
        q = 40
        
        if method.lower()=="shallow":
            model = spectNN.spectralNNShallow(N,d,M,L,setup.act_fn,setup.init)
            check_file = "Shallow_"+str(M)+"_"+str(L)+".pt"
        elif method.lower()=="deep":
            model = spectNN.spectralNNDeep(N,d,M,L,depth,width,setup.act_fn,setup.init)
            check_file = "Deep_"+str(M)+"_"+str(L)+"_"+str(depth)+"_"+str(width)+".pt"
        elif method.lower()=="deepshared1":
            model = spectNN.spectralNNDeepshared1(N,d,M,L,depth,width,setup.act_fn,setup.init)
            check_file = "Deepshared1_"+str(M)+"_"+str(L)+"_"+str(depth)+"_"+str(width)+".pt"
        elif method.lower()=="deepshared2":
            model = spectNN.spectralNNDeepshared2(N,d,M,L,depth,width,setup.act_fn,setup.init)
            check_file = "Deepshared2_"+str(M)+"_"+str(L)+"_"+str(depth)+"_"+str(width)+".pt"
        elif method.lower()=="deepshared3":
            model = spectNN.spectralNNDeepshared3(N,d,M,L,depth,width,setup.act_fn,setup.init)
            check_file = "Deepshared3_"+str(M)+"_"+str(L)+"_"+str(depth)+"_"+str(width)+".pt"
        else:
            exit("Undefined model specified! Aborting...")

In [ ]:
        optimizer = setup.optimizer(model.params,lr=setup.lr)
        loss = Ifn.loss_spectralNN(N, setup.wt_fn, grid_size=setup.loss_grid, q=q)
        print("Fitting the model ...")
        start_time = time.time()
        l_tr = Ifn.spectral_NN_optim(x,u,model,loss,optimizer,epochs=setup.epochs,checkpoint_file=check_file)
        time_ellapsed = time.time() - start_time
        print("Model fitted. Time taken: {} seconds" .format(time_ellapsed))

In [ ]:
        with torch.no_grad():
            num = loss.loss_fn(x,model(u)).item()
            den = loss.loss_fn(x,0*x).item()
            train_err = num/den
        print("Relative training error: {:.2f}%" .format(train_err*100))
        print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))

In [ ]:
        theta_file = dirc+"True_thetas"+str(repl+1)+".dat"
        loc_file = dirc+"True_locations"+str(repl+1)+".dat"
        spect_file = dirc+"True_spectrum"+str(repl+1)+".dat"

        spect_dens_est = Ifn.spectral_density_evaluation(model, q=q, wt_fn=setup.wt_fn)
        test_err,num,den,tr_cospect,tr_quadspect,err_cospect,err_quadspect = Ifn.spectral_error_computation(spect_dens_est,theta_file,loc_file,spect_file)

        print("Relative test error: {:.2f}%" .format(test_err*100))
        print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))
        print("Cospectra: Error - {:.4f}, Actual - {:.4f}" .format(err_cospect,tr_cospect))
        print("Quadspectra: Error - {:.4f}, Actual - {:.4f}" .format(err_quadspect,tr_quadspect))

In [ ]:
        emp_spect_dens = Ifn.empirical_spectral_density(x, u, q=q, wt_fn=setup.wt_fn)
        test_err,num,den,tr_cospect,tr_quadspect,err_cospect,err_quadspect = Ifn.emp_spectral_error_computation(emp_spect_dens,theta_file,loc_file,spect_file)

        print("Relative test error: {:.2f}%" .format(test_err*100))
        print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))
        print("Cospectra: Error - {:.4f}, Actual - {:.4f}" .format(err_cospect,tr_cospect))
        print("Quadspectra: Error - {:.4f}, Actual - {:.4f}" .format(err_quadspect,tr_quadspect))